In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df_raw= pd.read_csv('../10 data/rotten_tomatoes_movie_reviews.csv')

In [3]:
df = pd.read_csv('../10 data/rotten_tomatoes_movies.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143258 entries, 0 to 143257
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    143258 non-null  object 
 1   title                 142891 non-null  object 
 2   audienceScore         73248 non-null   float64
 3   tomatoMeter           33877 non-null   float64
 4   rating                13991 non-null   object 
 5   ratingContents        13991 non-null   object 
 6   releaseDateTheaters   30773 non-null   object 
 7   releaseDateStreaming  79420 non-null   object 
 8   runtimeMinutes        129431 non-null  float64
 9   genre                 132175 non-null  object 
 10  originalLanguage      129400 non-null  object 
 11  director              139041 non-null  object 
 12  writer                90116 non-null   object 
 13  boxOffice             14743 non-null   object 
 14  distributor           23001 non-null   object 
 15  

## 00 데이터 EDA

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444963 entries, 0 to 1444962
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1444963 non-null  object
 1   reviewId        1444963 non-null  int64 
 2   creationDate    1444963 non-null  object
 3   criticName      1444963 non-null  object
 4   isTopCritic     1444963 non-null  bool  
 5   originalScore   1009745 non-null  object
 6   reviewState     1444963 non-null  object
 7   publicatioName  1444963 non-null  object
 8   reviewText      1375738 non-null  object
 9   scoreSentiment  1444963 non-null  object
 10  reviewUrl       1234038 non-null  object
dtypes: bool(1), int64(1), object(9)
memory usage: 111.6+ MB


In [6]:
df_raw.nunique()

id                  69263
reviewId          1432569
creationDate         8510
criticName          15510
isTopCritic             2
originalScore        1729
reviewState             2
publicatioName       2707
reviewText        1359771
scoreSentiment          2
reviewUrl         1138350
dtype: int64

In [7]:
# 중복 제거 및 index 초기화
df_raw.drop_duplicates(inplace=True)
df_raw.reset_index(drop=True, inplace=True)

In [8]:
# 리뷰 개수 90% 인 기준 산출
df_raw['id'].value_counts().quantile(0.9)

np.float64(56.0)

In [9]:
# 리뷰가 56개 이상인 영화(id)의 index 추출
selected_index_1 = df_raw['id'].value_counts()[df_raw['id'].value_counts()>=56].index

In [10]:
selected_index_1

Index(['joker_2019', 'once_upon_a_time_in_hollywood', 'avengers_endgame',
       'captain_marvel', 'a_star_is_born_2018', 'black_panther_2018',
       'star_wars_the_rise_of_skywalker', 'the_batman', 'dune_2021',
       'avengers_infinity_war',
       ...
       'ginger_snaps', '1090089-jack_frost', 'test_pattern', 'claires_camera',
       'the_wound_2017', 'the_mad_womens_ball', 'days_of_heaven',
       'broken_2012', '1104207-panic',
       'jeanne_dielman_23_quai_du_commerce_1080_bruxelles'],
      dtype='object', name='id', length=6963)

In [11]:
# 추출된 Index로 df_raw에서 리뷰 데이터 추출
selected_df = df_raw[df_raw['id'].isin(selected_index_1)]

In [12]:
# index 초기화
selected_df.reset_index(drop=True, inplace= True)

In [13]:
selected_df

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,the_duff,2763233,2021-02-02,Richard Crouse,False,3/5,fresh,Richard Crouse,A school comedy so predictable the screenwrite...,POSITIVE,http://www.richardcrouse.ca/the-duff-3-stars-a...
1,the_duff,2692661,2020-05-27,Andrew Galdi,False,NaN,fresh,Movie Bitches,It was good acting and good writing.,POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
2,the_duff,2679627,2020-03-26,Avaryl Halley,False,NaN,fresh,Movie Bitches,"Oh dear, am I the Duff?",POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
3,the_duff,2615809,2019-08-15,Udita Jhunjhunwala,True,NaN,fresh,Livemint,While director Ari Sandel's adaptation is devo...,POSITIVE,https://www.livemint.com/Leisure/3wDs58QVt0Mtm...
4,the_duff,2586271,2019-05-14,Olivia Luder,True,3/5,fresh,One Room With A View,Not clever enough to be genre-defining in its ...,POSITIVE,https://oneroomwithaview.com/2015/03/06/duff-r...
...,...,...,...,...,...,...,...,...,...,...,...
925866,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
925867,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
925868,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
925869,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [14]:
# 리뷰를 4개이상한 리뷰어들의 index 추출
selected_index_2 = selected_df['criticName'].value_counts()[selected_df['criticName'].value_counts()>=4].index

In [15]:
selected_index_2

Index(['Frank Swietek', 'Roger Moore', 'Brian Orndorf', 'Jeffrey M. Anderson',
       'Rich Cline', 'Nell Minow', 'Laura Clifford', 'James Berardinelli',
       'Dennis Schwartz', 'David Nusair',
       ...
       'Anne Crump', 'Simon Guerrier', 'Valeriy Kolyadych',
       'Juan José Ontiveros', 'Megan Moffat', 'Mekeisha Madden Toby',
       'Hannah Chambers', 'Philip Horne', 'Jerrica Tisdale', 'Rob Beames'],
      dtype='object', name='criticName', length=6093)

In [16]:
# 추출된 Index로 selected_df에서 리뷰 데이터(final_df) 추출
final_df = selected_df[selected_df['criticName'].isin(selected_index_2)]
final_df

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,the_duff,2763233,2021-02-02,Richard Crouse,False,3/5,fresh,Richard Crouse,A school comedy so predictable the screenwrite...,POSITIVE,http://www.richardcrouse.ca/the-duff-3-stars-a...
1,the_duff,2692661,2020-05-27,Andrew Galdi,False,NaN,fresh,Movie Bitches,It was good acting and good writing.,POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
2,the_duff,2679627,2020-03-26,Avaryl Halley,False,NaN,fresh,Movie Bitches,"Oh dear, am I the Duff?",POSITIVE,https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...
3,the_duff,2615809,2019-08-15,Udita Jhunjhunwala,True,NaN,fresh,Livemint,While director Ari Sandel's adaptation is devo...,POSITIVE,https://www.livemint.com/Leisure/3wDs58QVt0Mtm...
4,the_duff,2586271,2019-05-14,Olivia Luder,True,3/5,fresh,One Room With A View,Not clever enough to be genre-defining in its ...,POSITIVE,https://oneroomwithaview.com/2015/03/06/duff-r...
...,...,...,...,...,...,...,...,...,...,...,...
925866,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
925867,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
925868,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
925869,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [17]:
final_df.reset_index(drop=True,inplace=True)

In [ ]:
# fresh 여부를 기준으로 데이터 split
X_train, X_test, Y_train, Y_test = train_test_split(final_df.drop('reviewState', axis=1), final_df['reviewState'], stratify=final_df.criticName, test_size=0.1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=X_train.criticName, test_size=0.25)

In [ ]:
X_train

# 01 MF 모델 구축해보기

## 1. 전처리 이후 df 를 MF 모델 구성에 맞게 구성
### 전처리가 끝나고 Train, test 나눈 이후

In [ ]:
final_df

In [ ]:
df_1 = final_df.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Fresh 여부를 숫자로 변환
df_1['freshness'] = df_1['reviewState'].map({'fresh': 1, 'rotten': 0})

# Fresh 여부가 nan인 경우 선호하지 않는 것으올 간주 -> 0으로 치환
df_1['freshness'].fillna(0,inplace=True)

# 사용자와 영화 ID를 Label Encoding
le_user = LabelEncoder()
le_movie = LabelEncoder()

df_1['user_id'] = le_user.fit_transform(df_1['criticName'])
df_1['movie_id'] = le_movie.fit_transform(df_1['id'])

# 필요한 열만 유지
data = df_1[['user_id', 'movie_id', 'freshness']]


train, test = train_test_split(data, test_size=0.2, random_state=42)

## 2. 유저-영화 행렬 생성

In [ ]:
import numpy as np

# 사용자-영화 매트릭스 생성
num_users = data['user_id'].nunique()
num_movies = data['movie_id'].nunique()

rating_matrix = np.zeros((num_users, num_movies))
for row in train.itertuples():
    rating_matrix[row.user_id, row.movie_id] = row.freshness


## 4. train 셋을 통해 MF 모델 학습


In [ ]:
from sklearn.metrics import mean_squared_error

# 사용자와 영화 잠재 행렬 초기화
latent_features = 10
U = np.random.rand(num_users, latent_features)
V = np.random.rand(num_movies, latent_features)

# 하이퍼파라미터 설정
learning_rate = 0.01
regularization = 0.1
epochs = 50

# 학습 루프
for epoch in range(epochs):
    for user in range(num_users):
        for movie in range(num_movies):
            error = rating_matrix[user, movie] - np.dot(U[user, :], V[movie, :])
            U[user, :] += learning_rate * (error * V[movie, :] - regularization * U[user, :])
            V[movie, :] += learning_rate * (error * U[user, :] - regularization * V[movie, :])

# 예측
predicted_matrix = np.dot(U, V.T)


## 5.Test셋 모델 테스트

In [ ]:
# Test 데이터에서 RMSE 계산
observed = []
predicted = []

for row in test.itertuples():
    obs = row.freshness
    pred = predicted_matrix[row.user_id, row.movie_id]
    observed.append(obs)
    predicted.append(pred)

rmse = np.sqrt(mean_squared_error(observed, predicted))
print(f"RMSE: {rmse}")


In [ ]:
min(predicted)

# GNNsage 사용

## 1. df graph 형식으로 바꾸기

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

# 1. 그래프 구조 시각화
def visualize_graph(data):
    """
    PyTorch Geometric 데이터를 NetworkX로 변환하여 그래프 시각화.
    """
    graph = to_networkx(data, to_undirected=True)
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(graph, seed=42)  # 노드 배치
    nx.draw(graph, pos, with_labels=False, node_size=20, node_color='blue', alpha=0.7)
    plt.title("Graph Visualization: Movies and Reviews", fontsize=16)
    plt.show()

In [19]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.metrics import accuracy_score

In [20]:

# 1. 데이터 로드
# Rotten Tomatoes 데이터셋 카피
df_2 = final_df.copy()
# 예시 데이터 구조: ['review_id', 'movie_id', 'review_text', 'fresh']

# 라벨 인코딩 (Fresh -> 1, Not Fresh -> 0)
df_2['fresh_label'] = df_2['reviewState'].map({'fresh': 1, 'rotten': 0})

In [28]:

# 2. 그래프 생성
# 노드와 엣지 정의
movie_ids = df_2['id'].unique()
review_ids = df_2['reviewId'].unique()

# 영화와 리뷰를 각각 노드로 정의
movie_idx = {movie_id: i for i, movie_id in enumerate(movie_ids)}
review_idx = {review_id: i + len(movie_ids) for i, review_id in enumerate(review_ids)}

# 엣지 리스트 생성 (리뷰 <-> 영화 관계)
edges = []
for _, row in df_2.iterrows():
    edges.append([review_idx[row['reviewId']], movie_idx[row['id']]])

In [29]:
# 엣지를 PyTorch 텐서로 변환
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# 노드 특징 (여기서는 임의로 임베딩을 초기화)
num_nodes = len(movie_ids) + len(review_ids)
x = torch.randn((num_nodes, 16))  # 16차원의 임베딩 벡터 초기화

# 라벨 (Fresh 여부)
y = torch.tensor(df_2['fresh_label'].values, dtype=torch.long)

# 3. PyTorch Geometric 데이터 구성
data = Data(x=x, edge_index=edge_index, y=y)

In [30]:
data

Data(x=[924378, 16], edge_index=[2, 917415], y=[917415])

In [31]:
# 4. 학습/테스트 데이터 분리
train_mask, test_mask = train_test_split(range(len(df_2)), test_size=0.2, random_state=42)
data.train_mask = torch.tensor(train_mask, dtype=torch.long)
data.test_mask = torch.tensor(test_mask, dtype=torch.long)


In [32]:
# 5. GraphSAGE 모델 정의
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [33]:
# 모델 초기화
model = GraphSAGE(in_channels=16, hidden_channels=32, out_channels=2)  # 2 클래스 (Fresh/Not Fresh)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


In [34]:
# 6. 모델 학습
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# 7. 테스트
def test(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[mask].max(1)[1]
        acc = accuracy_score(data.y[mask].cpu(), pred.cpu())
    return acc

In [35]:


# 학습 루프
for epoch in range(50):  # 50 에포크
    loss = train()
    train_acc = test(data.train_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

# 학습 완료 후 테스트 정확도 출력
print(f'Final Test Accuracy: {test_acc:.4f}')


Epoch 1, Loss: 0.7130, Train Acc: 0.5783, Test Acc: 0.5775
Epoch 2, Loss: 0.6780, Train Acc: 0.6489, Test Acc: 0.6471
Epoch 3, Loss: 0.6560, Train Acc: 0.6648, Test Acc: 0.6634
Epoch 4, Loss: 0.6460, Train Acc: 0.6665, Test Acc: 0.6651
Epoch 5, Loss: 0.6455, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 6, Loss: 0.6497, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 7, Loss: 0.6536, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 8, Loss: 0.6547, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 9, Loss: 0.6529, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 10, Loss: 0.6494, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 11, Loss: 0.6458, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 12, Loss: 0.6429, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 13, Loss: 0.6414, Train Acc: 0.6665, Test Acc: 0.6652
Epoch 14, Loss: 0.6412, Train Acc: 0.6664, Test Acc: 0.6650
Epoch 15, Loss: 0.6418, Train Acc: 0.6663, Test Acc: 0.6648
Epoch 16, Loss: 0.6427, Train Acc: 0.6662, Test Acc: 0.6648
Epoch 17, Loss: 0.6435, Train Acc: 0.6661, Test A

# test

In [36]:
df_2.columns

Index(['id', 'reviewId', 'creationDate', 'criticName', 'isTopCritic',
       'originalScore', 'reviewState', 'publicatioName', 'reviewText',
       'scoreSentiment', 'reviewUrl', 'fresh_label'],
      dtype='object')

In [37]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
import numpy as np

# Step 1: Load Rotten Tomato Data
# (Assume you have a CSV with 'user_id', 'movie_id', and 'freshness' columns)
df_3 = final_df.copy()

# Preprocess data
user_ids = df_3['criticName'].unique()
movie_ids = df_3['id'].unique()

# Map user and movie IDs to indices
user_mapping = {user_id: i for i, user_id in enumerate(user_ids)}
movie_mapping = {movie_id: i for i, movie_id in enumerate(movie_ids)}

df_3['user_idx'] = df_3['criticName'].map(user_mapping)
df_3['movie_idx'] = df_3['id'].map(movie_mapping)

# Encode freshness (1 for fresh, 0 for rotten)
df_3['freshness'] = df_3['reviewState'].apply(lambda x: 1 if x == 'fresh' else 0)

In [39]:


# Step 2: Create Heterogeneous Graph
graph = HeteroData()

# Add user and movie nodes
graph['user'].num_nodes = len(user_ids)
graph['movie'].num_nodes = len(movie_ids)

# Add edges with freshness as edge attributes
user_movie_edges = torch.tensor(df_3[['user_idx', 'movie_idx']].values.T, dtype=torch.long)
freshness = torch.tensor(df_3['freshness'].values, dtype=torch.float)

graph['user', 'rates', 'movie'].edge_index = user_movie_edges
graph['user', 'rates', 'movie'].edge_attr = freshness

# Add reverse edges for bidirectional connections
graph['movie', 'rated_by', 'user'].edge_index = user_movie_edges.flip(0)
graph['movie', 'rated_by', 'user'].edge_attr = freshness

In [40]:
# Step 3: Define GraphSAGE Model
class GraphSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x_user, x_movie, edge_index_user_movie, edge_index_movie_user):
        # User node embeddings
        x_user = self.conv1((x_user, x_movie), edge_index_user_movie)
        x_user = torch.relu(x_user)
        x_user = self.conv2((x_user, x_movie), edge_index_user_movie)

        # Movie node embeddings
        x_movie = self.conv1((x_movie, x_user), edge_index_movie_user)
        x_movie = torch.relu(x_movie)
        x_movie = self.conv2((x_movie, x_user), edge_index_movie_user)

        return x_user, x_movie

In [42]:
# Step 4: Initialize Model, Optimizer, and Loss
model = GraphSAGE(hidden_channels=64, out_channels=32)

# Randomly initialize user and movie node features
num_user_features = 64
num_movie_features = 64
x_user = torch.randn((len(user_ids), num_user_features), requires_grad=True)
x_movie = torch.randn((len(movie_ids), num_movie_features), requires_grad=True)

optimizer = torch.optim.Adam(list(model.parameters()) + [x_user, x_movie], lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [43]:
# Step 5: Train/Test Split
train_data, test_data = train_test_split(df_3, test_size=0.2, random_state=42)

# Extract train edges
train_edges = torch.tensor(train_data[['user_idx', 'movie_idx']].values.T, dtype=torch.long)
train_labels = torch.tensor(train_data['freshness'].values, dtype=torch.float)

In [44]:
# Step 6: Training Loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    x_user_embed, x_movie_embed = model(
        x_user, x_movie,
        graph['user', 'rates', 'movie'].edge_index,
        graph['movie', 'rated_by', 'user'].edge_index
    )

    # Compute loss on training edges
    user_embeds = x_user_embed[train_edges[0]]
    movie_embeds = x_movie_embed[train_edges[1]]
    preds = (user_embeds * movie_embeds).sum(dim=1)  # Dot product for interaction score
    loss = criterion(preds, train_labels)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Step 7: Movie Recommendation
def recommend_movies(user_id, top_k=10):
    user_idx = user_mapping[user_id]
    user_embed = x_user_embed[user_idx]

    scores = (user_embed * x_movie_embed).sum(dim=1)
    recommended_movie_indices = scores.argsort(descending=True)[:top_k]

    return [movie_ids[idx.item()] for idx in recommended_movie_indices]

# Example recommendation
user_id_to_recommend = user_ids[0]
recommended_movies = recommend_movies(user_id_to_recommend)
print("Recommended Movies:", recommended_movies)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6963x32 and 64x64)